In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import keras as kr


In [87]:
df_2023 = pd.read_csv('../election_resources/cleaned_2023_data.csv')
df_2023.head()

,question_id,end_date,pollster,sample_size,population,politician,favorable,unfavorable,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,results
0,178640,2023-08-08,YouGov,1500.0,a,Donald Trump,41.0,56.0,21.0,20.0,12.0,44.0,0
1,178641,2023-08-08,YouGov,1178.0,rv,Donald Trump,41.0,58.0,23.0,18.0,11.0,47.0,0
2,178646,2023-08-08,YouGov,1500.0,a,Joe Biden,44.0,53.0,20.0,24.0,13.0,40.0,0
3,178647,2023-08-08,YouGov,1178.0,rv,Joe Biden,45.0,53.0,23.0,22.0,11.0,42.0,0
4,178652,2023-08-08,YouGov,1500.0,a,Mike Pence,27.0,58.0,6.0,21.0,27.0,31.0,0


In [88]:
df_2023.drop(columns=['question_id', 'end_date'], inplace=True)
df_2023.fillna(0, inplace=True)
df_2023.head()

,pollster,sample_size,population,politician,favorable,unfavorable,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,results
0,YouGov,1500.0,a,Donald Trump,41.0,56.0,21.0,20.0,12.0,44.0,0
1,YouGov,1178.0,rv,Donald Trump,41.0,58.0,23.0,18.0,11.0,47.0,0
2,YouGov,1500.0,a,Joe Biden,44.0,53.0,20.0,24.0,13.0,40.0,0
3,YouGov,1178.0,rv,Joe Biden,45.0,53.0,23.0,22.0,11.0,42.0,0
4,YouGov,1500.0,a,Mike Pence,27.0,58.0,6.0,21.0,27.0,31.0,0


In [89]:
politicians_to_replace = list(df_2023['politician'].value_counts()[df_2023['politician'].value_counts() < 5].index)
for i in politicians_to_replace:
    df_2023['politician'] = df_2023['politician'].replace(i, 'Other')

In [90]:
pollster_to_replace = list(df_2023['pollster'].value_counts()[df_2023['pollster'].value_counts() < 8].index)
for i in pollster_to_replace:
    df_2023['pollster'] = df_2023['pollster'].replace(i, 'Other')

In [91]:
df_2023 = pd.get_dummies(df_2023, dtype=float)
df_2023.head()

,sample_size,favorable,unfavorable,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,results,pollster_AP-NORC,pollster_Beacon Research/Shaw & Company,...,politician_Joe Biden,politician_Larry A. Elder,politician_Marianne Williamson,politician_Mike Pence,politician_Nikki Haley,politician_Robert F. Kennedy,politician_Ron DeSantis,politician_Tim Scott,politician_Vivek G. Ramaswamy,politician_Will Hurd
0,1500.0,41.0,56.0,21.0,20.0,12.0,44.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1178.0,41.0,58.0,23.0,18.0,11.0,47.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1500.0,44.0,53.0,20.0,24.0,13.0,40.0,0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1178.0,45.0,53.0,23.0,22.0,11.0,42.0,0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1500.0,27.0,58.0,6.0,21.0,27.0,31.0,0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
df_2023.shape

(2379, 55)

In [93]:
nn_model = tf.keras.models.load_model('trained_model_president.h5')
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 75)                4200      
                                                                 
 dense_19 (Dense)            (None, 15)                1140      
                                                                 
 dense_20 (Dense)            (None, 67)                1072      
                                                                 
 dense_21 (Dense)            (None, 37)                2516      
                                                                 
 dense_22 (Dense)            (None, 61)                2318      
                                                                 
 dense_23 (Dense)            (None, 5)                 310       
                                                                 
 dense_24 (Dense)            (None, 29)               

In [94]:
scaler = StandardScaler()
X_scaler = scaler.fit(df_2023)
X_scaled = X_scaler.transform(df_2023)


In [95]:
predictions = nn_model.predict(X_scaled)

75/75 [==============================] - 1s 2ms/step


In [96]:
predictions

array([[0.5137716 ],
       [0.55157363],
       [0.48177132],
       ...,
       [0.5437341 ],
       [0.5482766 ],
       [0.48800048]], dtype=float32)